In [2]:
#parsing our scores into pandas dataframe which we can use in ML later
import os
import p
andas as pd
from bs4 import BeautifulSoup

In [3]:
SCORE_DIR = "data/scores"

In [4]:
box_scores = os.listdir(SCORE_DIR)

In [6]:
len(box_scores)

[]

In [ ]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]  #gives full path to box scores

In [ ]:
#we will do a little bit of cleanup before parsing the code by removing the column headings, etc

def parse_htlm(box_score):
    with open(box_score) as f:
        html = f.read() #read it into html variable
        
    soup = BeautifulSoup(html) #creating an instance to parse it
    [s.decompose() for s in soup.select("tr.over_header")] #selects pieces we want to remove (the column headings, etc) #returns a list
    [s.decompose() for s in soup.select("tr.thread")]
    return soup

In [ ]:
#reading the score from the data

def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0] # ID = line_score #returns dataframe
    cols = list(line_score.columns) #fixing the column names
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]] #the points are given as scores per quarter, hence we combine all to compare diff games with diff quarters
    return line_score

In [ ]:
#reading the stats

def  read_stats(soup, team, stats):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce") #convert columns to numeric values
    return df

In [ ]:
#to get the season number
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [ ]:
base_cols = None
games = []

box_score = box_scores[0]

for box_score in box_scores: 
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = [] #contains summary for both teams
    for team in teams:
        basic = read_stats(soup, team, "basic") #reading basic stats
        advanced = read_stats(soup, team, "advanced") #reading advanced stats
    
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]]) #to compress #concat last row of basic and advanced dataframe we do not need more data from one game
        totals.index = totals.index.str.lower() #converts column names in lower case
    
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()]) #to compress data, we take the player who scored highest
        maxes.index = maxes.index.str.lower() + "_max" #change index of maxes for pandas to treat them as separate columns
    
        summary = pd.concat([totals, maxes])
    
        #we have 100 stats for one game, 200 stats for another; we should have consistent stats
        if bas_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep = "first")) #removing duplicate values
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
    
        summaries.append(summary) #appending both the team summaries
    
    summary = pd.concat(summaries, axis=1).T #.T to transform the rows to columns

    game = pd.concat([summary, line_score], axis=1) #concat line_score column to summary

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index() #reversing the dataframe
    game_opp.columns += "_opp" #remaining the columns for the opposite team

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8] #to get the date (upto 8 chars)
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games)%100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

In [ ]:
games_df = pd.concat(games, ignore_index = True)

In [ ]:
games_df.to_csv("nba_games.csv")